In [1]:
!pip install pandas numpy fastparquet


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd

# Read roads csv file

In [3]:
# detailed_trajectory = pd.read_pickle("./data_with_trajectory_20s/courier_detailed_trajectory_20s.pkl.xz",compression="xz")
# road_df=pd.read_csv("../LaDe/road-network/roads.csv",sep="\t")

road_df=pd.read_csv("../processed/roads_translated_motherfile.csv")
print(road_df.head(3))

print(" ----------------------------------------------------------------------------- ")

print(" ----------------------------------------------------------------------------- ")

delivery = pd.read_csv("../LaDe/delivery/delivery_hz.csv")
print(delivery.head(3))

print(" ----------------------------------------------------------------------------- ")
print(" ----------------------------------------------------------------------------- ")







    osm_id  code     fclass  name  ref oneway  maxspeed  layer bridge tunnel  \
0  4296533  5141    service   NaN  NaN      B         0      0      F      F   
1  4296592  5115   tertiary   朝晖路  NaN      B         0      0      F      F   
2  4298707  5114  secondary  体育场路  NaN      F         0      0      F      F   

       city                                           geometry city_chinese  
0  Hangzhou  LINESTRING (13373499.140421594 3538916.2947984...          杭州市  
1  Hangzhou  LINESTRING (13376291.100042382 3539415.0794264...          杭州市  
2  Hangzhou  LINESTRING (13375063.92510783 3538697.18123532...          杭州市  
 ----------------------------------------------------------------------------- 
 ----------------------------------------------------------------------------- 
   order_id  region_id      city  courier_id        lng       lat  aoi_id  \
0    583722          0  Hangzhou         175  120.17895  30.26401     749   
1   2819059          0  Hangzhou         175  120.178

# reading all postmanid into a folder parquet from a single pcikle file 

In [4]:
trajectory20s = pd.read_pickle("../LaDe/data_with_trajectory_20s/courier_detailed_trajectory_20s.pkl.xz",compression="xz")

trajectory20s.head(3)

,ds,postman_id,gps_time,lat,lng
0,321,106f5ac22cfd1574b196d16fed62f90d,03-21 07:31:58,3.953700e+06,3.053400e+06
1,321,106f5ac22cfd1574b196d16fed62f90d,03-21 07:32:18,3.953700e+06,3.053398e+06
2,321,106f5ac22cfd1574b196d16fed62f90d,03-21 07:32:41,3.953700e+06,3.053398e+06


In [5]:
import lzma, pickle, os
from pathlib import Path
import pandas as pd
import tempfile
PARQUET_COMPRESSION = 'snappy' 

DF_VAR = 'trajectory20s'                      # replace if your DataFrame variable name differs
OUT_DIR = Path('../processed/per_postman_pickles')
OUT_DIR.mkdir(exist_ok=True)
            

In [6]:
# assume `df` is already in memory (loaded from pickle)
# ensure postman_id column exists and is string
df = globals().get(DF_VAR)
if df is None:
    raise SystemExit("DataFrame variable 'df' not found in globals(). Please assign your DataFrame to variable `df`.")

if 'postman_id' not in df.columns:
    raise SystemExit("Column 'postman_id' not found in DataFrame.")

df['postman_id'] = df['postman_id'].astype(str)

In [7]:
# find top 100 postmen by record count
top50 = df['postman_id'].value_counts().nlargest(50).index.tolist()


# write per-postman parquet files
for pid in top50:
    sub = df[df['postman_id'] == pid].copy()
    out_path = OUT_DIR / f'postman_{pid}.parquet'
    sub.to_parquet(out_path, index=False, compression=PARQUET_COMPRESSION)

print(f"Saved {len(top50)} parquet files to: {OUT_DIR.resolve()}")

Saved 100 parquet files to: /home/aymuos/Documents/Github/starship/processed/per_postman_pickles


In [8]:

# traj_df["gps_time"]=pd.to_datetime(traj_df["gps_time"])

# traj_gsort_df = (
#     traj_df.groupby("postman_id",group_keys=False)
#     .apply(lambda g:g.sort_values("gps_time"))
#     .reset_index(drop=True)
# )



# traj_basegroup_df = (
#     traj_df.groupby(["ds","postman_id"])
#     .size()
#     .reset_index(name="count_coordinates")
# )



In [9]:
road_df.columns

# osm_id = openstreetmap_id 
# 'code' = osm internal code schema 
# 'fclass', =  functional category of road -  motorway, primary, secondary, residential
#  'name', = official name availble in osm
#  'ref'   = Reference code (like highway numbers, e.g., “G60” for a national expressway). ex. S102 
#  'oneway', = Indicates if the road is one-way (yes/no). Critical for routing algorithms.
#  'maxspeed'= Maximum legal speed limit on the road segment (km/h).
#  'layer' = Vertical layer of the road (used when roads overlap, e.g., flyovers vs. tunnels) --> 
#  'bridge'= Boolean flag (yes/no) indicating if the segment is a bridge.
#  'tunnel'= Boolean flag (yes/no) indicating if the segment is a tunnel.
#  'city', = The city to which the road belongs (Shanghai, Hangzhou, Chongqing, Jilin, Yantai).
# 'geometry' = The polyline coordinates (latitude/longitude pairs) describing the exact shape of the road segment.

Index(['osm_id', 'code', 'fclass', 'name', 'ref', 'oneway', 'maxspeed',
       'layer', 'bridge', 'tunnel', 'city', 'geometry', 'city_chinese'],
      dtype='object')

# Messing with the trajectory data 

In [10]:
trajectory20s['ds'] = pd.to_datetime('2021-'+trajectory20s['ds'], format='%Y-%m-%d', errors='coerce')

trajectory20s.head(3)

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U5'), dtype('int64')) -> None

In [ ]:
# traj_gsort_df["seq"]=(
#     traj_gsort_df.groupby("postman_id")[["lat","lng"]]
#     .apply(lambda g: (g != g.shift())).any(axis=1).cumsum()
#     .reset_index(level=0, drop=True)
# )


In [ ]:
# traj_spacegroup_df = (
#     traj_gsort_df.groupby(["postman_id","seq"])
#     .agg(
#         lat = ("lat","first"),
#         lng = ("lng","first"),
#         start_time = ("gps_time","min"),
#         end_time = ("gps_time","max"),
#         n_pings = ("gps_time","size")
#     )
#     .reset_index()
# )

In [ ]:
# traj_spacegroup_df.to_pickle("trajectory_spacegroup.pkl")